In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
base_path = '../../'
sys.path.append(base_path)

In [10]:


from torch import nn
import torch

"""
3D U-Net: Learning Dense Volumetric Segmentation from Sparse Annotation
Paper URL: https://arxiv.org/abs/1606.06650
Author: Amir Aghdam
"""

class Conv3DBlock(nn.Module):
    """
    The basic block for double 3x3x3 convolutions in the analysis path
    -- __init__()
    :param in_channels -> number of input channels
    :param out_channels -> desired number of output channels
    :param bottleneck -> specifies the bottlneck block
    -- forward()
    :param input -> input Tensor to be convolved
    :return -> Tensor
    """

    def __init__(self, in_channels, out_channels, bottleneck = False) -> None:
        super(Conv3DBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels= in_channels, out_channels=out_channels//2, kernel_size=(3,3,3), padding=1)
        self.bn1 = nn.BatchNorm3d(num_features=out_channels//2)
        self.conv2 = nn.Conv3d(in_channels= out_channels//2, out_channels=out_channels, kernel_size=(3,3,3), padding=1, stride=1)
        self.bn2 = nn.BatchNorm3d(num_features=out_channels)
        self.relu = nn.ReLU()
        self.bottleneck = bottleneck
        if not bottleneck:
            self.pooling = nn.MaxPool3d(kernel_size=(2,2,2), stride=2)

    
    def forward(self, input):
        res = self.relu(self.bn1(self.conv1(input)))
        res = self.relu(self.bn2(self.conv2(res)))
        out = None
        if not self.bottleneck:
            out = self.pooling(res)
        else:
            out = res
        return out, res




class UpConv3DBlock(nn.Module):
    """
    The basic block for upsampling followed by double 3x3x3 convolutions in the synthesis path
    -- __init__()
    :param in_channels -> number of input channels
    :param out_channels -> number of residual connections' channels to be concatenated
    :param last_layer -> specifies the last output layer
    :param num_classes -> specifies the number of output channels for dispirate classes
    -- forward()
    :param input -> input Tensor
    :param residual -> residual connection to be concatenated with input
    :return -> Tensor
    """

    def __init__(self, in_channels, res_channels=0, last_layer=False, num_classes=None) -> None:
        super(UpConv3DBlock, self).__init__()
        assert (last_layer==False and num_classes==None) or (last_layer==True and num_classes!=None), 'Invalid arguments'
        self.upconv1 = nn.ConvTranspose3d(in_channels=in_channels, out_channels=in_channels, kernel_size=(2, 2, 2), stride=2)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm3d(num_features=in_channels//2)
        self.conv1 = nn.Conv3d(in_channels=in_channels+res_channels, out_channels=in_channels//2, kernel_size=(3,3,3), padding=(1,1,1))
        self.conv2 = nn.Conv3d(in_channels=in_channels//2, out_channels=in_channels//2, kernel_size=(3,3,3), padding=(1,1,1), stride=1)
        self.last_layer = last_layer
        if last_layer:
            self.conv3 = nn.Conv3d(in_channels=in_channels//2, out_channels=num_classes, kernel_size=(1,1,1))
            
        
    def forward(self, input, residual=None):
        out = self.upconv1(input)
        if residual!=None: out = torch.cat((out, residual), 1)
        out = self.relu(self.bn(self.conv1(out)))
        out = self.relu(self.bn(self.conv2(out)))
        if self.last_layer: out = self.conv3(out)
        return out
        



class UNet3D(nn.Module):
    """
    The 3D UNet model
    -- __init__()
    :param in_channels -> number of input channels
    :param num_classes -> specifies the number of output channels or masks for different classes
    :param level_channels -> the number of channels at each level (count top-down)
    :param bottleneck_channel -> the number of bottleneck channels 
    :param device -> the device on which to run the model
    -- forward()
    :param input -> input Tensor
    :return -> Tensor
    """
    
    def __init__(self, in_channels, num_classes, level_channels=[64, 128, 256, 512], bottleneck_channel=1024) -> None:
        super(UNet3D, self).__init__()
        level_1_chnls, level_2_chnls, level_3_chnls, level_4_chnls = level_channels[0], level_channels[1], level_channels[2], level_channels[3]
        self.a_block1 = Conv3DBlock(in_channels=in_channels, out_channels=level_1_chnls)
        self.a_block2 = Conv3DBlock(in_channels=level_1_chnls, out_channels=level_2_chnls)
        self.a_block3 = Conv3DBlock(in_channels=level_2_chnls, out_channels=level_3_chnls)
        self.a_block4 = Conv3DBlock(in_channels=level_3_chnls, out_channels=level_4_chnls)
        self.bottleNeck = Conv3DBlock(in_channels=level_4_chnls, out_channels=bottleneck_channel, bottleneck= True)
        self.s_block4 = UpConv3DBlock(in_channels=bottleneck_channel, res_channels=level_4_chnls)
        self.s_block3 = UpConv3DBlock(in_channels=level_4_chnls, res_channels=level_3_chnls)
        self.s_block2 = UpConv3DBlock(in_channels=level_3_chnls, res_channels=level_2_chnls)
        self.s_block1 = UpConv3DBlock(in_channels=level_2_chnls, res_channels=level_1_chnls, num_classes=num_classes, last_layer=True)

    
    def forward(self, input):
        #Analysis path forward feed
        out, residual_level1 = self.a_block1(input)
        out, residual_level2 = self.a_block2(out)
        out, residual_level3 = self.a_block3(out)
        out, residual_level4 = self.a_block4(out)
        out, _ = self.bottleNeck(out)

        #Synthesis path forward feed
        out = self.s_block4(out, residual_level4)
        out = self.s_block3(out, residual_level3)
        out = self.s_block2(out, residual_level2)
        out = self.s_block1(out, residual_level1)
        return out
model = UNet3D(in_channels=1, num_classes=5, level_channels=[16, 32, 64,128], bottleneck_channel=256)

In [11]:
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
#ds_pv = PyvistaFlowFieldDataset.load_from_huggingface(num_samples=1000, data_dir='datasets/pyvista')
ds_pv= PyvistaFlowFieldDataset.try_from_directory("datasets/pyvista",1000)
ds_pv[800].plot_surface("WallShearStress_0")

2025-05-25 07:45:44.730 ( 169.062s) [    7359D3195740]     vtkLookupTable.cxx:162    ERR| vtkLookupTable (0x5cca76419aa0): Bad table range: [inf, -inf]
ERROR:root:Bad table range: [inf, -inf]
ERROR:root:Bad table range: [inf, -inf]
2025-05-25 07:45:44.736 ( 169.067s) [    7359D3195740]     vtkLookupTable.cxx:162    ERR| vtkLookupTable (0x5cca76419aa0): Bad table range: [inf, -inf]
ERROR:root:Bad table range: [inf, -inf]
2025-05-25 07:45:44.737 ( 169.068s) [    7359D3195740]     vtkLookupTable.cxx:162    ERR| vtkLookupTable (0x5cca76419aa0): Bad table range: [inf, -inf]


Widget(value='<iframe src="http://localhost:40591/index.html?ui=P_0x735870fddd90_1&reconnect=auto" class="pyvi…

In [ ]:
from src.voxel_flow_field_dataset import VoxelFlowFieldDataset, VoxelFlowFieldDatasetConfig
ds_voxel = VoxelFlowFieldDataset('datasets/voxels',VoxelFlowFieldDatasetConfig(ds_pv, resolution=(32,16,16)))
#ds_voxel = VoxelFlowFieldDataset('datasets/voxels')
ds_voxel.normalize()

Computing bounds:   0%|          | 0/1000 [00:00<?, ?it/s]2025-05-25 07:45:47.592 ( 171.923s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de13f00): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:45:47.592 ( 171.923s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de13f00): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-25 07:45:47.592 ( 171.923s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de13f00): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.
2025-05-25 07:45:47.596 ( 171.927s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de13f00): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:45:47.597 ( 171.928s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de13f00): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-25 

Unloading sample to free memory


2025-05-25 07:45:57.884 ( 182.215s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de1e9f0): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:45:57.884 ( 182.215s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de1e9f0): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-25 07:45:57.884 ( 182.215s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7de1e9f0): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.
2025-05-25 07:45:58.495 ( 182.826s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7783e800): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:45:58.495 ( 182.826s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7783e800): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.


Unloading sample to free memory


2025-05-25 07:45:58.495 ( 182.826s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7783e800): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.
2025-05-25 07:45:59.082 ( 183.413s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7e5e87f0): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:45:59.082 ( 183.413s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca7e5e87f0): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.


...

2025-05-25 07:46:10.880 ( 195.211s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca76124e70): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-25 07:46:10.880 ( 195.212s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca76124e70): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.


Unloading sample to free memory


2025-05-25 07:46:11.493 ( 195.824s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca761247c0): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-25 07:46:11.493 ( 195.824s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca761247c0): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-25 07:46:11.493 ( 195.824s) [    7359D3195740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x5cca761247c0): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.


KeyboardInterrupt: 

: 

In [ ]:

import torch.utils.data as data
dataloader = data.DataLoader(ds_voxel.get_default_loadable_dataset(), batch_size=32, shuffle=True)
len(ds_voxel)

20

In [31]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)
criterion = nn.MSELoss()
lr=2e-5
epochs=800
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.999)

losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for x,y in dataloader:
        x, y = x.to(device), y.to(device)
        x=x.float().unsqueeze(-1)
        x = x.permute(0, 4, 1, 2, 3)  # (N, D, H, W, C) → (N, C, D, H, W)
        y = y.permute(0, 4, 1, 2, 3)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    scheduler.step()
    losses.append(epoch_loss / len(dataloader))
    if (epoch) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

# Save the model
torch.save(model.state_dict(), 'unet3d.pth')

Using device: cuda
Epoch 1/800, Loss: 0.9595879912376404
Epoch 11/800, Loss: 0.9578304290771484
Epoch 21/800, Loss: 0.9543054103851318
Epoch 31/800, Loss: 0.951217770576477
Epoch 41/800, Loss: 0.9483742117881775
Epoch 51/800, Loss: 0.945498526096344
Epoch 61/800, Loss: 0.9426692128181458
Epoch 71/800, Loss: 0.9398363828659058
Epoch 81/800, Loss: 0.9370134472846985
Epoch 91/800, Loss: 0.9341943860054016
Epoch 101/800, Loss: 0.9313877820968628
Epoch 111/800, Loss: 0.9285998940467834
Epoch 121/800, Loss: 0.9258338212966919
Epoch 131/800, Loss: 0.9230915904045105
Epoch 141/800, Loss: 0.920382559299469
Epoch 151/800, Loss: 0.9176996350288391
Epoch 161/800, Loss: 0.9150489568710327
Epoch 171/800, Loss: 0.912429690361023
Epoch 181/800, Loss: 0.9098362922668457
Epoch 191/800, Loss: 0.9072781205177307
Epoch 201/800, Loss: 0.904751718044281
Epoch 211/800, Loss: 0.9022565484046936
Epoch 221/800, Loss: 0.8997911810874939
Epoch 231/800, Loss: 0.8973562121391296
Epoch 241/800, Loss: 0.89495319128036

KeyboardInterrupt: 

In [7]:
model.load_state_dict(torch.load('unet3d.pth'))

<All keys matched successfully>

In [8]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.show()

NameError: name 'losses' is not defined

In [32]:
mask,Y = next(iter(dataloader))
mask = mask.to(device)
Y = Y.to(device)

In [33]:
x=mask.float().unsqueeze(-1)
x = x.permute(0, 4, 1, 2, 3)  # (N, D, H, W, C) → (N, C, D, H, W)
y = y.permute(0, 4, 1, 2, 3)
y_pred = model(x)
y_pred = y_pred.permute(0, 2, 3, 4, 1)  # (N, C, D, H, W) → (N, D, H, W, C)


In [34]:
pred_sample = ds_voxel.prediction_to_sample(mask[0], y_pred[0])

In [35]:
pred_sample.plot("Pressure")

Widget(value='<iframe src="http://localhost:36941/index.html?ui=P_0x7d5071e32210_0&reconnect=auto" class="pyvi…

In [36]:
true_sample = ds_voxel.prediction_to_sample(mask[0], Y[0])
true_sample.plot("Pressure")

Widget(value='<iframe src="http://localhost:36941/index.html?ui=P_0x7d4f2728ae90_1&reconnect=auto" class="pyvi…

In [39]:
pred_sample.plot_slice_interactively("Pressure",axis="z")

interactive(children=(IntSlider(value=8, description='slice_idx', max=15), Output()), _dom_classes=('widget-in…

In [40]:
true_sample.plot_slice_interactively("Pressure", axis="z")

interactive(children=(IntSlider(value=8, description='slice_idx', max=15), Output()), _dom_classes=('widget-in…